In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import BlipProcessor, BlipForQuestionAnswering, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from PIL import Image
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm
from transformers import BitsAndBytesConfig

# Download NLTK data
nltk.download('punkt')

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define dataset class
class VQADataset(Dataset):
    def __init__(self, df, processor, dataset_dir):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.dataset_dir = dataset_dir

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.dataset_dir, row['path'])

        try:
            image = Image.open(img_path).convert("RGB")
        except:
            return None

        question = str(row['Question']).strip() if pd.notnull(row['Question']) else ""
        answer = str(row['Answer']).strip() if pd.notnull(row['Answer']) else ""

        question += " (answer in one word)"

        inputs = self.processor(
            images=image,
            text=question,
            return_tensors="pt",
            padding="max_length",
            max_length=32,
            truncation=True
        )

        answer_inputs = self.processor(
            text=answer,
            return_tensors="pt",
            padding="max_length",
            max_length=32,
            truncation=True
        )

        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs["labels"] = answer_inputs["input_ids"].squeeze(0)

        return inputs

# Load and prepare data
df = pd.read_csv('/kaggle/input/actual-dataset/merged_data_actual.csv')
dataset_dir = '/kaggle/input/images/abo-images-small/images/small'

# Take a subset of 10k images
df = df.sample(n=10000, random_state=42).reset_index(drop=True)

# Split into train and eval (90%/10%)
train_df = df.sample(frac=0.9, random_state=42)
eval_df = df.drop(train_df.index)

# Initialize processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

# Define quantization configuration for base model
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# Load base model with quantization
model = BlipForQuestionAnswering.from_pretrained(
    "Salesforce/blip-vqa-base",
    quantization_config=quantization_config,
    device_map="auto"
)

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "value", "vision_encoder.encoder.layer.*.attention.attention"],
    lora_dropout=0.05,
    bias="none"
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Create datasets
train_dataset = VQADataset(train_df, processor, dataset_dir)
eval_dataset = VQADataset(eval_df, processor, dataset_dir)

# Filter out None values
train_dataset = [x for x in train_dataset if x is not None]
eval_dataset = [x for x in eval_dataset if x is not None]

# Training arguments
training_args = TrainingArguments(
    output_dir="./blip-vqa-lora",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    fp16=True,
    save_steps=1000,
    save_total_limit=2,
    eval_strategy="steps",
    eval_steps=500,
    logging_steps=100,
    learning_rate=1e-4,
    weight_decay=0.01,
    warmup_steps=500,
    remove_unused_columns=False,
    report_to="none"
)

# Define compute_metrics function
def compute_metrics(p):
    predictions, labels = p
    decoded_preds = processor.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
    decoded_labels = processor.batch_decode(labels, skip_special_tokens=True)

    smoothie = SmoothingFunction().method1
    bleu_scores = []
    exact_matches = []

    for pred, label in zip(decoded_preds, decoded_labels):
        reference = [label.lower().split()]
        candidate = pred.lower().split()
        bleu = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0), smoothing_function=smoothie)
        bleu_scores.append(bleu)
        exact_matches.append(label.strip().lower() == pred.strip().lower())

    return {
        "bleu": np.mean(bleu_scores),
        "accuracy": np.mean(exact_matches),
    }

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained("./blip-vqa-lora-final")
processor.save_pretrained("./blip-vqa-lora-final")

# Evaluate after training
final_results = trainer.evaluate()
print("Final evaluation results:", final_results)

# Generate predictions on test set
test_results = []
bleu_scores = []
smoothie = SmoothingFunction().method1
model.eval()

for i in tqdm(range(len(eval_df))):
    row = eval_df.iloc[i]
    img_path = os.path.join(dataset_dir, row['path'])

    try:
        image = Image.open(img_path).convert("RGB")
    except:
        continue

    question = str(row['Question']).strip() if pd.notnull(row['Question']) else ""
    true_answer = str(row['Answer']).strip() if pd.notnull(row['Answer']) else ""

    question += " (answer in one word)"

    inputs = processor(image, question, return_tensors="pt").to(device)
    output = model.generate(**inputs)
    pred = processor.decode(output[0], skip_special_tokens=True).strip()

    reference = [true_answer.lower().split()]
    candidate = pred.lower().split()
    bleu = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0), smoothing_function=smoothie)
    bleu_scores.append(bleu)

    test_results.append({
        'Question': question,
        'GroundTruth': true_answer,
        'Prediction': pred,
        'BLEU-1': bleu,
        'Match': true_answer.strip().lower() == pred.strip().lower()
    })

# Convert to DataFrame and evaluate
test_res_df = pd.DataFrame(test_results)
accuracy = test_res_df['Match'].mean()
average_bleu = sum(bleu_scores) / len(bleu_scores)

# Save and show results
test_res_df.to_csv('/kaggle/working/blip_vqa_lora_finetuned_results.csv', index=False)
print(f"LoRA Fine-tuned Model Exact Match Accuracy: {accuracy:.4f}")
print(f"LoRA Fine-tuned Model Average BLEU-1 Score: {average_bleu:.4f}")
print(test_res_df.head(10))